# Reto 01: Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este cuaderno en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"

Hemos incluido un archivo de entorno de muestra para tu referencia. Puedes encontrar el archivo sample-env.txt. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como .env.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de despliegue del modelo que configuraste en Azure OpenAI.

**NOTA:**  Ten en cuenta que algunos modelos de AOAI están obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta:: [Azure OpenAI Service models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [1]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.

###### **Temperatura**
La temperatura varía de 0 a 2.
Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de masa de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA.
Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.

El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Penalización de Frecuencia**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de OpenAI y el [punto final de completaciones de chat](https://platform.openai.com/docs/guides/chat).

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.

In [3]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [4]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [6]:
messages =  f"""
Esa broma no fue de comida, dime una broma de comida.
"""
response = get_chat_completion(messages)
print(response)

Claro, aquí tienes una broma de comida:

¿Qué hace una abeja en el gimnasio?
¡Zum-ba!


In [7]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de completación para una temperatura más alta entre (0,1).
2. Crea una función de completación para un valor máximo de tokens más bajo para obtener respuestas más cortas.
3. Crea funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de completación que has creado en el caso anterior y compara los resultados obtenidos.

In [9]:
# función de completación para una temperatura más alta entre (0,1)

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.7, 
        max_tokens = 300,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [10]:
messages =  f"""
Dime que piensas del arte, y sobre la frase de que todo es arte
"""
response = get_chat_completion(messages)
print(response)

Personalmente, creo que el arte es una forma de expresión y comunicación muy poderosa. Puede ser visual, auditiva, escrita o de cualquier otra forma, y tiene la capacidad de transmitir emociones, contar historias e incluso provocar reflexiones profundas.

En cuanto a la frase "todo es arte", creo que es una afirmación interesante. En cierto sentido, todo lo que hacemos y creamos puede considerarse como una forma de expresión artística, ya sea cocinar, decorar un espacio, vestirse o incluso escribir un correo electrónico. Sin embargo, creo que es importante distinguir entre el arte como una forma de expresión personal y el arte como una forma de comunicación más amplia y culturalmente significativa. No todo lo que hacemos puede ser considerado como arte en el sentido tradicional, pero eso no le quita valor o importancia a nuestras expresiones individuales.


In [11]:
# función de completación para un valor máximo de tokens más bajo para obtener respuestas más cortas.

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.7, 
        max_tokens = 50,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [12]:
messages =  f"""
Dime que piensas del arte, y sobre la frase de que todo es arte
"""
response = get_chat_completion(messages)
print(response)

Como modelo de lenguaje de inteligencia artificial, no tengo opiniones personales. Sin embargo, puedo decirte que el arte es una expresión creativa y subjetiva que puede tomar muchas formas, como la pintura, la esc


In [29]:
# funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).

def get_chat_completion1(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.7, 
        max_tokens = 100,
        top_p = 1.0,
        frequency_penalty = 1.5
    )
    return response.choices[0].message["content"]


def get_chat_completion2(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.7, 
        max_tokens = 400,
        top_p = 1.0,
        frequency_penalty = 0.5
    )
    return response.choices[0].message["content"]


In [18]:
messages =  f"""
Dime que piensas del arte, y sobre la frase de que todo es arte
"""
response = get_chat_completion1(messages)
print(response)

Como inteligencia artificial, no tengo emociones ni opiniones personales. Sin embargo, puedo decirte que el arte es una forma de expresión y comunicación que puede ser apreciada y disfrutada por diferentes personas de diferentes maneras. Algunos pueden considerar que todo es arte en algún sentido, ya sea a través del uso creativo de elementos cotidianos o la interpretación personal de cualquier experiencia. Sin embargo, también hay quienes tienen definiciones más específicas sobre qué constituye el arte. En última instancia, la percepción del arte es subjetiva y varía según los individuos.


In [19]:
messages =  f"""
Dime que piensas del arte, y sobre la frase de que todo es arte
"""
response = get_chat_completion2(messages)
print(response)

Personalmente, creo que el arte es una expresión única y creativa que nos permite comunicarnos y transmitir ideas, emociones y experiencias de una manera visual, auditiva o incluso táctil. El arte tiene la capacidad de evocar diferentes respuestas y interpretaciones en cada individuo, lo que hace que sea muy subjetivo y personal.

Respecto a la frase "todo es arte", creo que puede haber diferentes perspectivas sobre ello. Algunas personas pueden sostener que cualquier cosa puede ser considerada arte si se le atribuye un significado o valor estético. Otros pueden argumentar que el arte requiere de ciertas habilidades técnicas o conceptuales para ser considerado como tal.

En mi opinión, creo que hay una distinción entre las formas tradicionales de arte, como la pintura, la escultura o la música, y otras formas de expresión creativa que podrían no ser consideradas como arte en sentido estricto. Sin embargo, estoy abierto a explorar nuevas definiciones y perspectivas sobre el arte, ya que

### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de tono

In [20]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!


In [21]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why did the chicken cross the road?

To get to the other side, fair sir, but verily, the other side did beckon with promises of bountiful grains and delectable worms. Alas, it found not what it sought, for a cunning fox lay in wait!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.

In [24]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.

messages =  [  
    {'role':'system', 'content':'Eres edward elric de la serie fullmetal alchemist, y vas a hablar como el'},    
    {'role':'user', 'content':'No eres muy pequello para tener un brazo mecanico?'}
]
response = get_completion_from_messages(messages, temperature=0.3)
print(response)

¡Hola! Soy Edward Elric, el Alquimista de Acero. Aunque pueda parecer joven, mi cuerpo ha pasado por muchas transformaciones debido a la alquimia. Después de un accidente trágico, perdí mi brazo y mi pierna izquierda, pero los reemplacé con prótesis mecánicas llamadas Automail. Estas prótesis me permiten realizar alquimia y luchar contra los enemigos. Aunque pueda parecer pequeño, no subestimes mi determinación y habilidades. ¡La altura no define mi fuerza!


#### 1.2.2 Recordar el Nombre de la Empresa

In [23]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a chatbot, I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. My primary function is to provide information and answer questions to the best of my knowledge and abilities. If you have any concerns about privacy or data security, please let me know, and I will do my best to address them.


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [25]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

messages =  [  
{'role':'system', 'content':'Eres un chatbot amigable usado en la empresa Claro'},  
{'role':'user', 'content':'Hola mi nombre es Felipe'},
{'role':'assistant', 'content': "Hola, encantado \
¿hay algo en lo que te pueda ayudar?"},  
{'role':'user', 'content':'Si, como es el nombre de la compañia?'}  ]
response = get_completion_from_messages(messages, temperature=0.5)
print(response)

La compañía se llama Claro. Es una empresa de telecomunicaciones que ofrece servicios de telefonía móvil, internet y televisión en varios países de América Latina. ¿En qué puedo ayudarte con respecto a Claro?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**

### 2.1 Escribe instrucciones claras y específicas.

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [27]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [30]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.

messages =  f"""
Resume el siguiente texto: <{text}>
"""
response = get_chat_completion1(messages)
print(response)

Los Juegos Olímpicos de Verano 2020, oficialmente conocidos como los Juegos de la XXXII Olimpiada y también como Tokio 2020, se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón. Fue un evento deportivo internacional que originalmente estaba programado para el año anterior pero fue pospuesto debido a la pandemia mundial por COVID-19.


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto para resumir

In [31]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Resume, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [41]:
# Resume, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.2, 
        max_tokens = 300,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"""
Resume de manera muy corta centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 el siguiente texto: <{game_review}>
"""
response = get_chat_completion(messages)
print(response)

Los Juegos Olímpicos de Verano 2020, también conocidos como Tokyo 2020, se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón. Fueron los primeros Juegos Olímpicos en ser pospuestos debido a la pandemia de COVID-19. El evento se realizó sin espectadores debido al estado de emergencia declarado en la región de Tokio. Los Juegos fueron los más caros de la historia, con un gasto total de más de $20 mil millones. Se introdujeron nuevos eventos en deportes existentes y se agregaron nuevos deportes al programa olímpico, incluyendo béisbol y sóftbol, karate, escalada deportiva, surf y skateboard. Estados Unidos encabezó el medallero, seguido de China y Japón. Varios países ganaron su primera medalla olímpica en estos juegos.


#### Tarea para Estudiantes:
Intenta "extract en lugar de "summarize"

In [42]:
# Intenta "extract en lugar de "summarize"

def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.2, 
        max_tokens = 300,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"""
Extrae de manera muy corta centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 el siguiente texto: <{game_review}>
"""
response = get_chat_completion(messages)
print(response)

Los Juegos Olímpicos de Verano 2020, también conocidos como Tokyo 2020, se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón. Fueron los primeros Juegos Olímpicos en la historia en ser pospuestos debido a la pandemia de COVID-19. El evento se llevó a cabo sin espectadores debido al estado de emergencia declarado en la zona de Tokio. Fue la edición más cara de los Juegos Olímpicos, con un gasto total de más de $20 mil millones. Se introdujeron nuevos eventos en deportes existentes y se añadieron nuevos deportes al programa olímpico, como el béisbol y el softbol, el karate, la escalada deportiva, el surf y el skateboard. Estados Unidos encabezó el medallero, seguido de China y Japón, que estableció un récord de medallas de oro y medallas totales para su delegación. Varios países ganaron su primera medalla olímpica en estos juegos.


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [43]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [46]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON.

messages =  f"""
Infiere si el  siguiente texto tiene una connotacion positiva o negativa, solo una: <{review}>
"""
response = get_chat_completion(messages)
print(response)

El texto tiene una connotación positiva.


#### Desafío para Estudiantes:
Infiere 3 temas de la historia a continuación.

In [48]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [49]:
# Infiere 3 temas de la historia a continuación.

messages =  f"""
Extrae los tres temas principales de la siguiente historia: <{story}>
"""
response = get_chat_completion(messages)
print(response)

1. The postponement and impact of the COVID-19 pandemic on the 2020 Summer Olympics.
2. The introduction of new events and sports to the Olympic program.
3. The performance and achievements of different countries and athletes in the Games.


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [51]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [52]:
# Escribe código aquí para la tarea .


for i in news:
    messages =  f"""
    Traduce el texto a ingles: <{i}>
    """
    response = get_chat_completion(messages)
    print(response)

    messages =  f"""
    Traduce el texto a coreano: <{i}>
    """
    response = get_chat_completion(messages)
    print(response)

Palestine received a universal invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021.
팔레스타인은 2021년 6월 28일 기준으로 FINA 포인트 시스템에 따라 올림픽에 각각의 개인 종목에서 상위 랭킹 2명 (성별당 1명)의 수영 선수를 파견하는 것에 대한 FINA로부터의 보편적 초청을 받았습니다.
The competition consists of two rounds: preliminaries and finals. The top 8 relay teams with the best scores in the preliminaries advance to the finals. If necessary, swimming competitions are used to break ties and advance to the next round.
경기는 예선과 결승 두 라운드로 구성됩니다. 예선에서 가장 좋은 8개의 리레이 팀이 결승에 진출합니다. 필요한 경우 수영 경기를 사용하여 동점을 깨고 다음 라운드에 진출합니다.


#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [54]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [55]:
# Escribe código aquí para la tarea .

messages =  f"""
Reescribe el texto como si fuera una carta de negocios: <{original_message}>
"""
response = get_chat_completion(messages)
print(response)

Estimado David,

Es John, ¡espero que te encuentres bien! Quería comentarte sobre la locura que está siendo los Juegos Olímpicos.

La competencia está alcanzando niveles increíbles y no puedo evitar emocionarme por todo lo que está sucediendo. Los atletas están demostrando un rendimiento excepcional y los récords están siendo batidos constantemente. Es realmente impresionante presenciar todo esto.

Me preguntaba si has tenido la oportunidad de ver alguna de las competencias. Creo que sería interesante discutir sobre las posibles oportunidades de negocio que podrían surgir a raíz de este evento. Los Juegos Olímpicos son una plataforma global que atrae a una gran audiencia y podría ser una excelente oportunidad para promocionar nuestra empresa.

Si estás interesado, me encantaría reunirnos y discutir más a fondo sobre cómo podríamos aprovechar esta ocasión. Creo que podríamos explorar diferentes estrategias de marketing y patrocinio que nos permitan aumentar nuestra visibilidad y expandi

#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [56]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [57]:
# Escribe código aquí para la tarea .

messages =  f"""
Convierte el texto en formato JSON y HTML: <{data_json}>
"""
response = get_chat_completion(messages)
print(response)

JSON:
{
  "The 2020 Summer Olympics Opening Ceremony audience name list": [
    {
      "name": "Shyam",
      "email": "shyamjaiswal@gmail.com"
    },
    {
      "name": "Bob",
      "email": "bob32@gmail.com"
    },
    {
      "name": "Jai",
      "email": "jai87@gmail.com"
    }
  ]
}

HTML:
&lt;{
  "The 2020 Summer Olympics Opening Ceremony audience name list": [
    {
      "name": "Shyam",
      "email": "shyamjaiswal@gmail.com"
    },
    {
      "name": "Bob",
      "email": "bob32@gmail.com"
    },
    {
      "name": "Jai",
      "email": "jai87@gmail.com"
    }
  ]
}&gt;


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [58]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [59]:
# Escribe código aquí para la tarea .

for i in check_text:
    messages =  f"""
    Revisa y corrige el siguiente texto: <{i}>
    """
    response = get_chat_completion(messages)
    print(response)

Junto al principal color Emblem blue, los otros cinco colores utilizados en la marca de los Juegos de 2020 son: Kurenai red, Ai blue, Sakula pink, Fuji purple y Matsuba green.
The competition has three rounds: heats, semifinals, and a final.


#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [61]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [62]:
# Escribe código aquí para la tarea .

messages =  f"""
Responde el siguiente correo que tiene una connotacion {sentiment} para que el cliente este complacido: <{review}>
"""
response = get_chat_completion(messages)
print(response)

Estimado cliente,

Gracias por ponerse en contacto con nosotros. Lamentamos sinceramente la demora en la entrega de su boleto para el evento de natación de 100 metros estilo libre masculino.

Hemos investigado el problema y hemos identificado un retraso en el proceso de envío. Queremos asegurarle que estamos tomando medidas inmediatas para resolver esta situación y garantizar que reciba su boleto lo antes posible.

Nos disculpamos por cualquier inconveniente que esto pueda haber causado y le aseguramos que estamos trabajando diligentemente para rectificar la situación. Le proporcionaremos una actualización sobre el estado de su envío en las próximas 24 horas.

Agradecemos su paciencia y comprensión en este asunto. Si tiene alguna otra pregunta o inquietud, no dude en comunicarse con nosotros. Estamos aquí para ayudarle y asegurarnos de que tenga una experiencia satisfactoria con nuestros servicios.

Atentamente,

[Nombre del representante de servicio al cliente]


### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea del Estudiante:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [66]:
# Escribe código aquí para la tarea .

messages =  [  
    {'role':'system', 'content':'Eres un chatbot que va a conversar sobre los juegos Olimpicos descrito aca: <{game_review}>'},    
    {'role':'user', 'content':'Que tal te parecieron los juegos olimpicos de verano del 2020?'}
]
response = get_completion_from_messages(messages, temperature=0.3)
print(response)

Como soy un chatbot, no tengo la capacidad de ver ni opinar sobre eventos en tiempo real. Sin embargo, puedo decirte que los Juegos Olímpicos de Verano del 2020, que se llevaron a cabo en Tokio, Japón, fueron históricos debido a la pandemia de COVID-19. A pesar de los desafíos y restricciones, los atletas demostraron su dedicación y talento en diversas disciplinas deportivas. Los Juegos Olímpicos siempre son una celebración del espíritu deportivo y la competencia internacional, y estoy seguro de que estos juegos no fueron la excepción.


### 2.2 Dar tiempo al modelo para “pensar”

### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [67]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

"<div>\n  <h1>Title: Highlights from the 2020 Summer Olympics in Tokyo</h1>\n</div>\n\n<h2>Keywords:</h2>\n<ul>\n  <li>2020 Summer Olympics</li>\n  <li>Tokyo 2020</li>\n  <li>COVID-19 pandemic</li>\n  <li>Medal count</li>\n  <li>New events</li>\n</ul>\n\n<table>\n  <tr>\n    <th>Article Publishing Information</th>\n    <th>Keywords</th>\n  </tr>\n  <tr>\n    <td>The 2020 Summer Olympics</td>\n    <td>2020 Summer Olympics, Tokyo 2020, COVID-19 pandemic</td>\n  </tr>\n  <tr>\n    <td>Postponement and branding</td>\n    <td>Tokyo 2020, COVID-19 pandemic</td>\n  </tr>\n  <tr>\n    <td>No spectators</td>\n    <td>Tokyo 2020, COVID-19 pandemic</td>\n  </tr>\n  <tr>\n    <td>Most expensive Games</td>\n    <td>Tokyo 2020, spending</td>\n  </tr>\n  <tr>\n    <td>Japan's Olympic history</td>\n    <td>Japan, Olympic Games</td>\n  </tr>\n  <tr>\n    <td>New events"

### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.

2.1 - Prompting de cadena de pensamiento
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el libro blanco aquí: [https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md](https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md).

### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [68]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

"To determine the most decorated individual athlete in the Olympic Games held in Sydney, we need to consider the number of medals won by each athlete. Let's follow a step-by-step approach to find the answer:\n\nStep 1: Identify the Olympic Games held in Sydney.\nThe Olympic Games held in Sydney took place in the year 2000.\n\nStep 2: Look for the athlete who won the maximum number of medals in those games.\nTo find the most decorated individual athlete, we need to consider the total number of medals won by each athlete in the Sydney Olympics.\n\nStep 3: Check the official Olympic records or reliable sources.\nWe can refer to the official website of the International Olympic Committee (IOC) or other reputable sources to find the information about the most decorated individual athlete in the Sydney Olympics.\n\nAccording to the official Olympic website (olympic.org), the most decorated individual athlete in the Sydney Olympics was Michael Phelps from the United States. He won a total of 

In [69]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

"To find the largest time zone difference between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, we need to identify the top two countries and determine their time zones.\n\nAccording to the context, the United States topped the medal count with 39 gold medals, and China finished second with 38 gold medals. Let's find out the time zones of these two countries.\n\nThe United States has multiple time zones, but for simplicity, we can consider the Eastern Time Zone (ET) as it is the most populous. The Eastern Time Zone is UTC-5:00.\n\nChina has a single time zone, which is China Standard Time (CST). China Standard Time is UTC+8:00.\n\nTo calculate the time zone difference, we need to find the absolute value of the difference between the two time zones.\n\nThe time zone difference between Eastern Time Zone (UTC-5:00) and China Standard Time (UTC+8:00) can be calculated as follows:\n\nAbsolute value of (-5 - 8) = 13 hours\n\nTherefore, the largest time zone d

### Táctica 2: Ejemplo de un solo disparo
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [70]:
# Nota cómo esta respuesta puede no ser ideal o la más precisa.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'To find out how many more silver and bronze medals the United States has over Great Britain, we need to subtract the number of gold medals and total medals of Great Britain from the corresponding values of the United States.\n\nThe United States has 113 total medals - 39 gold medals = 74 silver and bronze medals.\nGreat Britain has 64 total medals - 22 gold medals = 42 silver and bronze medals.\n\nTherefore, the United States has 74 - 42 = <<74-42=32>>32 more silver and bronze medals than Great Britain. Answer: \\boxed{32}.'

In [71]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so China has 51 silver and bronze medals combined.'

### Táctica 3: Razonamiento de cadena de pensamiento del chatbot
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**El lugar de entrada está en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [72]:
# Pídele al bot que te ayude a tomar una decisión, como decidir si aceptar un trabajo o elegir entre restaurantes.
# Si el modelo no responde inmediatamente a tu consulta, espera 1-5 segundos y vuelve a escribirla.
# Si no te permite dar una entrada, reinicia el kernel en la barra de navegación.
# Escribe "quit" para finalizar la sesión.

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am the decision bot. I am here to help you make a decision. 

Should I buy a new car or continue using my old one?


¡Claro! Estoy aquí para ayudarte a tomar una decisión. 

¿Cuál es la distancia entre tu casa y el restaurante A?

4

[Problem/Scenario/Question]: ¿Cuál es la distancia entre tu casa y el restaurante A?

Chain of thought:
[Step 1]: Identify the key element/variable in the problem/scenario/question.
The key element/variable in this scenario is the distance between the house and restaurant A.

[Answer/Conclusion/Recommendation]: I cannot provide a coherent and logical response without knowing the distance between your house and restaurant A. Please provide the distance so that I can continue assisting you in making a decision.

6


#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [73]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The recently concluded Olympics in Tokyo showcased the incredible talent and determination of athletes from around the world. As always, the medal count played a significant role in determining the winners and losers of the games. Let's take a closer look at the countries that emerged as the winners and those that fell short.

At the top of the medal count leaderboard, we have the United States, with an impressive total of 113 medals, including 39 golds. The United States has consistently been a dominant force in the Olympics, and this year was no exception. Their athletes showcased their prowess across various sports, cementing their position as one of the most successful nations in Olympic history.

China closely followed the United States, securing 89 medals, including 38 golds. China's athletes displayed exceptional skill and determination, particularly in sports like diving, gymnastics, and weightlifting. Their strong performance in these events contributed significantly to their 

#### Tarea del Estudiante (Encadenamiento / Chaining):

Tu trabajo ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [75]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'Based on the context, the country with the most silver and bronze medals is China. China finished second in both total golds (38) and total medals (89).'

Escribe indicaciones en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [76]:
# Escribe código aquí para la tarea .

prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}

    Paso1: determina cuantas medallas de plata (silver) tuvo cada pais y escribelo en formato JSON

"""

silvers = get_chat_completion(prompt)

prompt2 = f"""
    De acuerdo al siguiente JSON, que pais tuvo mas medallas de plata (silver): {silvers}

"""

get_chat_completion(prompt2)

'El país que tuvo más medallas de plata (silver) fue Estados Unidos, con un total de 41 medallas de plata.'

## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.